In this Analysis we are going to find the hyperparameters for training the LDA model. 

We are going to use topic coherence for this

#### What is topic coherence?

Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic. These measurements help distinguish between topics that are semantically interpretable topics and topics that are artifacts of statistical inference. But,

#### What is coherence?

A set of statements or facts is said to be coherent, if they support each other. Thus, a coherent fact set can be interpreted in a context that covers all or most of the facts. An example of a coherent fact set is "the game is a team sport", "the game is played with a ball", "the game demands great physical efforts"


### Coherence Measures

1. `C_v` measure is based on a sliding window, one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity
2. `C_p` is based on a sliding window, one-preceding segmentation of the top words and the confirmation measure of Fitelson's coherence
3. `C_uci` measure is based on a sliding window and the pointwise mutual information (PMI) of all word pairs of the given top words
4. `C_umass` is based on document cooccurrence counts, a one-preceding segmentation and a logarithmic conditional probability as confirmation measure
5. `C_npmi` is an enhanced version of the C_uci coherence using the normalized pointwise mutual information (NPMI)
6. `C_a` is baseed on a context window, a pairwise comparison of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosine similarity

Pre processing the text

In [1]:
### 1. Import Libraries

# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import numpy for matrix operation
import numpy as np

# Importing Gensim
import gensim
from gensim import corpora

# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

nlp = spacy.load('en_core_web_sm')

In [2]:
# Importing modules
import pandas as pd
import os
import sys

# Read data into papers
#papers = pd.read_csv(os.path.join(sys.path[0],'../Dataset/ClinicalSTS2018/ClinicalSTS/clinicalSTS.train.txt'))
ClinicalTexts = pd.read_csv(os.path.join(sys.path[0],'data/clinicalSTS.train.txt'), sep='\t', header=None)

# combining all the documents into a list:

corpus = ClinicalTexts[0].tolist()

In [3]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(corpus))

print(data_words[:1][0][:30])

['insulin', 'nph', 'human', 'novolin', 'unit', 'ml', 'suspension', 'subcutaneous', 'as', 'directed', 'by', 'prescriber']


In [4]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [5]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    #print(sent)
    doc = nlp(" ".join(sent.split()))
    print(doc)
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]


In [6]:
# Apply Preprocessing on the Corpus

# stop loss words 
stop = set(stopwords.words('english'))
stop.update(['from', 'subject', 're', 'edu', 'use'])

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # Form Bigrams
    data_words_bigrams = ''.join(bigram_mod[doc])
    print(data_words_bigrams)
    lem = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    print(lem)
    
    data_lemmatized = " ".join(lem)

    # remove punctuations + normalize the text
    #normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return data_lemmatized

# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus] 

Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.
Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.
['unit', 'suspension', 'subcutaneous', 'direct']
 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.
Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.
['arrive', 'ambulatory', 'gait', 'steady', 'history', 'obtain', 'patient', 'appear', 'comfortable', 'patient', 'cooperative', 'alert', 'skin', 'warm']
 Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.
Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.
['site', 'establish', 'right', 'forearm', 'use', 'gauge', 'catheter', 'attempt']
 No: new confusion or inability to stay alert and awake

Patient Education: Ready to learn, no apparent learning barriers were identified; learning preferences include listening.
['ready', 'learn', 'apparent', 'learning', 'barrier', 'identify', 'learning', 'preference', 'include', 'listen']
 Negative cardiovascular review of systems, Historian denies chest pain, dyspnea on exertion.
Negative cardiovascular review of systems, Historian denies chest pain, dyspnea on exertion.
['negative', 'cardiovascular', 'review', 'system', 'deny', 'chest', 'pain', 'dyspnea', 'exertion']
 No: joint swelling; pain of a type other than joint pain; a limp without known injury; discomfort in legs that improves with movement usually occurring at night; painful cramping in your hip, thigh or calf muscles after walking or climbing stairs; muscle cramps; lower leg swelling (without injury); varicose veins or 'spider veins'; ulcers or sores on the lower leg; red or hot area of the skin; pale, cold, or blue leg(s)or toes; nail problem; cracking, peeling, burning or it

When ambulating with a cane, place the cane in the hand on the side opposite the surgical limb.
['when', 'ambulate', 'cane', 'place', 'cane', 'hand', 'side', 'surgical', 'limb']
 Discussed goals, risk, alternatives, advance directives, and the necessity of the other members of the surgical team participating in the procedure with the patient (and whoever else is present).
Discussed goals, risk, alternatives, advance directives, and the necessity of the other members of the surgical team participating in the procedure with the patient (and whoever else is present).
['discuss', 'goal', 'risk', 'alternative', 'advance', 'directive', 'necessity', 'other', 'member', 'surgical', 'team', 'participate', 'procedure', 'patient', 'else', 'present']
No: fever present (greater than or equal to 100.4 F or 38 C) or suspected fever; shaking chills or nausea or vomiting present
No: fever present (greater than or equal to 100.4 F or 38 C) or suspected fever; shaking chills or nausea or vomiting present


Discussed risks, goals, alternatives, advance directives, and the necessity of other members of the healthcare team participating in the procedure with (patient) (legal representative and others present during the discussion).
['discuss', 'risk', 'goal', 'alternative', 'advance', 'directive', 'necessity', 'other', 'member', 'healthcare', 'team', 'participate', 'procedure', 'patient', 'legal', 'representative', 'other', 'present', 'discussion']
 Patient needs assistance with the following instrumental activities of daily  living: meal preparation, medication administration, housekeeping, shopping, managing finances, transportation use (drive car/use taxi/bus).
Patient needs assistance with the following instrumental activities of daily living: meal preparation, medication administration, housekeeping, shopping, managing finances, transportation use (drive car/use taxi/bus).
['patient', 'need', 'assistance', 'follow', 'instrumental', 'activity', 'daily', 'living', 'meal', 'preparation', 

No: new confusion or inability to stay alert and awake; noisy, wheezy or raspy breathing that does not clear with coughing; newly stiff or painful neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash); feeling like you are going to pass out EVERY time you stand (or sit) up or muffled voice or inability to open mouth fully
['new', 'confusion', 'inability', 'stay', 'alert', 'awake', 'noisy', 'wheezy', 'breathing', 'clear', 'cough', 'newly', 'stiff', 'painful', 'neck', 'purple', 'red', 'rash', 'blotch', 'stay', 'when', 'press', 'glass', 'feel', 'go', 'pass', 'time', 'stand', 'sit', 'up', 'muffled', 'voice', 'inability', 'open', 'mouth', 'fully']
 Patient discharged to home, ambulating without assistance, family driving, accompanied by husband/wife/partner, Discharge instructions given to patient, Above person(s) verbalized understanding of discharge instructions and follow-up care.
Patient discharged to home, ambulating without assistance, family driving, acc

...

The general care of a sprain includes the use of a medication to reduce pain, the use of a splint to reduce movement and Resting, Icing, Compressing and Elevating the injured area.
['general', 'care', 'include', 'use', 'medication', 'reduce', 'pain', 'use', 'splint', 'reduce', 'movement', 'elevate', 'injure', 'area']
 Discussed the risks, goals, alternatives, and the necessity of other members of the healthcare team participating in the procedure.
Discussed the risks, goals, alternatives, and the necessity of other members of the healthcare team participating in the procedure.
['discuss', 'risk', 'goal', 'alternative', 'necessity', 'other', 'member', 'healthcare', 'team', 'participate', 'procedure']
 All questions were answered, and the patient was provided with the following printed materials:
All questions were answered, and the patient was provided with the following printed materials:
['question', 'answer', 'patient', 'provide', 'follow', 'print', 'material']
Education provided on 

Patient Education: Cognitive and/or language difficulties or the patient's age prevented the patient from understanding the diagnosis and/or treatment plan.
['language', 'difficulty', 'patient', 'age', 'prevent', 'patient', 'understand', 'diagnosis', 'treatment', 'plan']
 I reviewed the history, examined the patient, reviewed the available imaging studies, and agree with the findings as presented.
I reviewed the history, examined the patient, reviewed the available imaging studies, and agree with the findings as presented.
['review', 'history', 'examine', 'patient', 'review', 'available', 'imaging', 'study', 'agree', 'finding', 'present']
 No: new confusion or inability to stay alert and awake; currently feeling like you are going to collapse every time you stand or sit up; complete inability to swallow; new neck pain and difficulty bending the neck; purple or red rash/blotches that stay when pressed by a glass (purpuric rash); pain, pressure or tightness in the chest, arm or shoulder,

I explained the diagnosis and treatment plan for the patient who expressed understanding of the content.
['explain', 'diagnosis', 'treatment', 'plan', 'patient', 'express', 'understanding', 'content']
Triamcinolone Acetonide [KENALOG] 0.1 % cream 1 apply topically as directed by prescriber as needed.
Triamcinolone Acetonide [KENALOG] 0.1 % cream 1 apply topically as directed by prescriber as needed.
['%', 'cream', 'apply', 'topically', 'direct', 'need']
 No: inability to speak or make normal sounds; new confusion or inability to stay alert and awake; complete inability to swallow; sudden onset of cough, choking or gagging due to inhaling something into your airway; pain, pressure or tightness in the chest, arm or shoulder, jaw, or neck; large amounts of pink or white frothy sputum or noisy, wheezy or raspy breathing that does not clear with coughing
No: inability to speak or make normal sounds; new confusion or inability to stay alert and awake; complete inability to swallow; sudden on

No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); any chest pain or discomfort; vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
['new', 'confusion', 'inability', 'stay', 'alert', 'awake', 'currently', 'struggle', 'breathe', 'even', 'inactive', 'rest', 'currently', 'feel', 'go', 'collapse', 'time', 'stand', 'sit', 'chest', 'pain', 'discomfort', 'vomit', 'look', 'ground', 'coffee', 'vomit', 'blood', 'uncontrollable', 'continuous', 'rectal', 'bleeding', 'black', 'sticky', 'tar', 'like', 'stool', 'purple', 'red', 'rash', 'blotch', 'stay', 'when', 'press', 'glass']
 No: new confusion or inability to stay alert and awake; noisy, wheezy or raspy breathing that does not clear with coughin

Green or yellow discharge in the eye or on eyelashes continuing throughout the day, bloodshot or pink eyes
['green', 'yellow', 'discharge', 'eye', 'eyelash', 'continue', 'day', 'bloodshot', 'pink', 'eye']
Patient is here for the following immunization(s):  Live Intranasal Influenza Virus Vaccination
Patient is here for the following immunization(s): Live Intranasal Influenza Virus Vaccination
['patient', 'here', 'follow', 'live']
 The above was discussed with the patient, and she voiced understanding of the content and plan.
The above was discussed with the patient, and she voiced understanding of the content and plan.
['above', 'discuss', 'patient', 'voice', 'understanding', 'content', 'plan']
 Side rails up, Cart/Stretcher in lowest position, Call light within reach, Hospital ID band on, Patient safety, dignity, sense of well being and individual rights are respected.
Side rails up, Cart/Stretcher in lowest position, Call light within reach, Hospital ID band on, Patient safety, digni

In [7]:
clean_corpus

[['unit', 'suspension', 'subcutaneous', 'direct'],
 ['arrive',
  'ambulatory',
  'gait',
  'steady',
  'history',
  'obtain',
  'patient',
  'appear',
  'comfortable',
  'patient',
  'cooperative',
  'alert',
  'skin',
  'warm'],
 ['site',
  'establish',
  'right',
  'forearm',
  'use',
  'gauge',
  'catheter',
  'attempt'],
 ['new',
  'confusion',
  'inability',
  'stay',
  'alert',
  'awake',
  'currently',
  'struggle',
  'breathe',
  'even',
  'inactive',
  'rest',
  'currently',
  'feel',
  'go',
  'collapse',
  'time',
  'stand',
  'sit',
  'vomit',
  'look',
  'ground',
  'coffee',
  'vomit',
  'blood',
  'uncontrollable',
  'continuous',
  'rectal',
  'bleeding',
  'black',
  'sticky',
  'tar',
  'like',
  'stool',
  'heavy',
  'vaginal',
  'bleeding',
  'purple',
  'red',
  'rash',
  'blotch',
  'stay',
  'when',
  'press',
  'glass'],
 ['spend',
  'minute',
  'patient',
  'great',
  '%',
  'time',
  'spend',
  'counsel',
  'patient',
  'regard',
  'diagnosis',
  'available',


Creating Document term Matrix

In [8]:
dict_ = corpora.Dictionary(clean_corpus)

print(dict_)

# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in clean_corpus]
doc_term_matrix[0:2]

Dictionary(1349 unique tokens: ['direct', 'subcutaneous', 'suspension', 'unit', 'alert']...)


[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1)]]

Building Base model

In [9]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                       id2word=dict_,
                                       num_topics=6, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [10]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())


[(0,
  '0.062*"patient" + 0.044*"use" + 0.036*"barrier" + 0.034*"teaching" + '
  '0.033*"discharge" + 0.029*"need" + 0.029*"understanding" + 0.027*"learning" '
  '+ 0.022*"verbalize" + 0.021*"include"'),
 (1,
  '0.052*"other" + 0.045*"procedure" + 0.045*"discuss" + 0.039*"member" + '
  '0.039*"participate" + 0.039*"team" + 0.037*"necessity" + 0.033*"risk" + '
  '0.030*"healthcare" + 0.028*"alternative"'),
 (2,
  '0.042*"patient" + 0.039*"treatment" + 0.038*"plan" + 0.035*"learning" + '
  '0.029*"diagnosis" + 0.026*"explain" + 0.026*"understanding" + '
  '0.025*"express" + 0.024*"learn" + 0.024*"barrier"'),
 (3,
  '0.037*"pain" + 0.018*"patient" + 0.012*"right" + 0.012*"change" + '
  '0.011*"leg" + 0.011*"time" + 0.010*"eye" + 0.008*"leave" + 0.007*"history" '
  '+ 0.007*"mcg"'),
 (4,
  '0.026*"stay" + 0.024*"inability" + 0.023*"new" + 0.016*"awake" + '
  '0.015*"alert" + 0.015*"confusion" + 0.014*"neck" + 0.013*"when" + '
  '0.013*"fever" + 0.012*"pain"'),
 (5,
  '0.017*"stay" + 0.017*

In [11]:
num_topics=6

import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dict_)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

#pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
pyLDAvis.save_html(LDAvis_prepared, os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics)+'.html'))

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.255045  0.008598       1        1  27.625419
4      0.201317  0.089172       2        1  22.619197
1      0.046286 -0.281402       3        1  14.329941
2     -0.171908  0.084612       4        1  12.814970
5      0.120782  0.096634       5        1  12.329507
3      0.058567  0.002385       6        1  10.280965, topic_info=          Term        Freq       Total Category  logprob  loglift
178   learning  129.000000  129.000000  Default  30.0000  30.0000
171    barrier  140.000000  140.000000  Default  29.0000  29.0000
182   teaching  101.000000  101.000000  Default  28.0000  28.0000
106  procedure   89.000000   89.000000  Default  27.0000  27.0000
24         use  147.000000  147.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
14        skin    6.664253   24.567307   Topic6  -5.1192   0.9702
552        day    6.728607   29.140358   Topic6  -5.1096   0.8091
114     review    6.422538   36.358642   Topic6  -5.1561   0.5413
386      daily    5.781800   18.609336   Topic6  -5.2612   1.1059
173  discharge    5.876498  115.049412   Topic6  -5.2450  -0.6995

[350 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
778       5  0.850405          #
279       1  0.951452  accompany
531       6  0.695915  actuation
157       5  0.650668  addiction
157       6  0.162667  addiction
...     ...       ...        ...
406       2  0.032912       will
406       3  0.197474       will
406       4  0.065825       will
406       5  0.394948       will
517       3  0.784105     worsen

[581 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 2, 3, 6, 4])

#### Compute Model Perplexity and Coherence Score

Let's calculate the baseline coherence score

In [12]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=clean_corpus, dictionary=dict_, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score:  0.5597837127472782


** **
#### Step 6: Hyperparameter tuning
** **
First, let's differentiate between model hyperparameters and model parameters :

- `Model hyperparameters` can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

- `Model parameters` can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters: 
- Number of Topics (K)
- Dirichlet hyperparameter alpha: Document-Topic Density
- Dirichlet hyperparameter beta: Word-Topic Density

We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use `C_v` as our choice of metric for performance comparison 

In [13]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=clean_corpus, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
compute_coherence_values(clean_corpus, dict_, k=6, a=0.1, b=0.1)


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.5451247971357184

In [15]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(clean_corpus)
print(num_of_docs)
corpus_sets = [gensim.utils.ClippedCorpus(clean_corpus, int(num_of_docs*0.75)), 
               clean_corpus]
#print(corpus_sets)

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dict_, 
                                                  k=k, a=a, b=b)
                    
                    print(cv)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv(os.path.join(sys.path[0],'results/lda_cliSTS.csv'), index=False)
    pbar.close()

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 0/540 [00:00<?, ?it/s]

750


  0%|          | 1/540 [00:01<12:00,  1.34s/it]

0.33440397034412667


  0%|          | 2/540 [00:02<13:36,  1.52s/it]

...

0.5667476693916741


100%|█████████▉| 539/540 [17:23<00:01,  1.86s/it]

0.541978471327607


100%|██████████| 540/540 [17:24<00:00,  1.93s/it]

0.5354133276993271


**Building a finetuned LDA model**

In [41]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=doc_term_matrix,
                                       id2word=dict_,
                                       num_topics=3, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                        alpha=0.91,
                                           eta=0.01,
                                       per_word_topics=True)


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=clean_corpus, dictionary=dict_, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Coherence Score:  0.7053012058661957


In [43]:
num_topics=3

import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dict_)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

#pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
pyLDAvis.save_html(LDAvis_prepared, os.path.join(sys.path[0],'results/ldavis_med_'+str(num_topics)+'.html'))

LDAvis_prepared

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.278790 -0.025964       1        1  40.520332
2      0.165052 -0.198610       2        1  34.437525
1      0.113738  0.224573       3        1  25.042143, topic_info=              Term        Freq       Total Category  logprob  loglift
104          other  121.000000  121.000000  Default  30.0000  30.0000
171        barrier  150.000000  150.000000  Default  29.0000  29.0000
106      procedure   97.000000   97.000000  Default  28.0000  28.0000
98         discuss  103.000000  103.000000  Default  27.0000  27.0000
149  understanding  143.000000  143.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
447        symptom   17.532484   32.473542   Topic3  -5.0421   0.7682
273        present   17.195479   36.611739   Topic3  -5.0616   0.6289
180           need   20.113887  117.196253   Topic3  -4.9048  -0.3778
60            time   17.827966  106.528910   Topic3  -5.0254  -0.4030
113           pain   17.129716  108.246067   Topic3  -5.0654  -0.4590

[243 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
170       1  0.965916  address
170       2  0.018575  address
170       3  0.018575  address
271       1  0.047273  advance
271       3  0.945451  advance
...     ...       ...      ...
64        1  0.019792     when
64        2  0.851047     when
64        3  0.138543     when
589       3  0.987572  whimper
486       1  1.025584     wife

[371 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [44]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())


[(0,
  '0.067*"patient" + 0.033*"use" + 0.033*"barrier" + 0.031*"understanding" + '
  '0.030*"learning" + 0.024*"discharge" + 0.024*"teaching" + 0.022*"include" + '
  '0.022*"need" + 0.021*"learn"'),
 (1,
  '0.039*"other" + 0.032*"discuss" + 0.032*"procedure" + 0.026*"member" + '
  '0.026*"team" + 0.025*"participate" + 0.025*"necessity" + 0.023*"risk" + '
  '0.020*"healthcare" + 0.019*"alternative"'),
 (2,
  '0.025*"stay" + 0.023*"new" + 0.022*"inability" + 0.021*"pain" + '
  '0.016*"alert" + 0.015*"confusion" + 0.015*"awake" + 0.014*"currently" + '
  '0.011*"when" + 0.011*"feel"')]


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
doc_term_matrix[:2]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1)]]

In [46]:
lda_model[doc_term_matrix]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
# printing the topic associations with the documents
count = 0
for i in lda_model[doc_term_matrix]:
    print("doc : ",count,i[0])
    count += 1

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


doc :  0 [(0, 0.13541345), (1, 0.7292853), (2, 0.13530125)]
doc :  1 [(0, 0.86751235), (1, 0.06511459), (2, 0.06737312)]
doc :  2 [(0, 0.13948801), (1, 0.77361965), (2, 0.08689232)]
doc :  3 [(0, 0.019438937), (1, 0.02002216), (2, 0.9605389)]
doc :  4 [(0, 0.834053), (1, 0.093739), (2, 0.07220796)]
doc :  5 [(0, 0.04698486), (1, 0.057746787), (2, 0.8952684)]
doc :  6 [(0, 0.019861491), (1, 0.020468049), (2, 0.9596705)]
doc :  7 [(0, 0.074879974), (1, 0.058431193), (2, 0.8666888)]
doc :  8 [(0, 0.07229211), (1, 0.8555613), (2, 0.07214657)]
doc :  9 [(0, 0.25130394), (1, 0.091371834), (2, 0.6573242)]
doc :  10 [(0, 0.25886497), (1, 0.668568), (2, 0.07256701)]
doc :  11 [(0, 0.123919316), (1, 0.7782868), (2, 0.09779382)]
doc :  12 [(0, 0.024151996), (1, 0.025235193), (2, 0.95061284)]
doc :  13 [(0, 0.06377462), (1, 0.8804511), (2, 0.055774212)]
doc :  14 [(0, 0.85502905), (1, 0.06916985), (2, 0.075801104)]
doc :  15 [(0, 0.89995164), (1, 0.048461422), (2, 0.051587)]
doc :  16 [(0, 0.72427

doc :  481 [(0, 0.89962083), (1, 0.04695894), (2, 0.053420234)]
doc :  482 [(0, 0.8385539), (1, 0.08063582), (2, 0.08081024)]
doc :  483 [(0, 0.4109949), (1, 0.07976704), (2, 0.50923806)]
doc :  484 [(0, 0.12049876), (1, 0.60750526), (2, 0.27199593)]
doc :  485 [(0, 0.47422886), (1, 0.16945182), (2, 0.35631934)]
doc :  486 [(0, 0.82347786), (1, 0.087693974), (2, 0.08882819)]
doc :  487 [(0, 0.025118291), (1, 0.9247246), (2, 0.050157137)]
doc :  488 [(0, 0.13541345), (1, 0.7292853), (2, 0.13530126)]
doc :  489 [(0, 0.8872844), (1, 0.0564316), (2, 0.056283988)]
doc :  490 [(0, 0.5793665), (1, 0.2500644), (2, 0.17056914)]
doc :  491 [(0, 0.085065424), (1, 0.08026722), (2, 0.8346674)]
doc :  492 [(0, 0.026339987), (1, 0.028253572), (2, 0.9454065)]
doc :  493 [(0, 0.5136964), (1, 0.37879047), (2, 0.10751314)]
doc :  494 [(0, 0.15798639), (1, 0.7236074), (2, 0.11840624)]
doc :  495 [(0, 0.85043466), (1, 0.09035423), (2, 0.059211068)]
doc :  496 [(0, 0.014299135), (1, 0.015099704), (2, 0.9706

In [50]:
# printing the topic associations with the documents
import operator
lstTopicsCorpus =[]

count = 0
for i in lda_model[doc_term_matrix]:
    print("doc : ",count, i[0])
    maxTopic = max(i[0],key=operator.itemgetter(1))[0]
    lstTopicsCorpus.append(maxTopic)
    count += 1

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


doc :  0 [(0, 0.13541345), (1, 0.7292853), (2, 0.13530125)]
doc :  1 [(0, 0.86751527), (1, 0.06511471), (2, 0.067369975)]
doc :  2 [(0, 0.1394901), (1, 0.77361757), (2, 0.086892314)]
doc :  3 [(0, 0.019438935), (1, 0.0200222), (2, 0.96053886)]
doc :  4 [(0, 0.8340282), (1, 0.09376368), (2, 0.072208144)]
doc :  5 [(0, 0.046984926), (1, 0.057766918), (2, 0.8952482)]
doc :  6 [(0, 0.019861491), (1, 0.020468207), (2, 0.9596703)]
doc :  7 [(0, 0.07488253), (1, 0.058431175), (2, 0.8666863)]
doc :  8 [(0, 0.07229209), (1, 0.8555614), (2, 0.072146565)]
doc :  9 [(0, 0.25130403), (1, 0.09137217), (2, 0.6573238)]
doc :  10 [(0, 0.25887954), (1, 0.668554), (2, 0.07256638)]
doc :  11 [(0, 0.12391972), (1, 0.7782865), (2, 0.097793825)]
doc :  12 [(0, 0.024151988), (1, 0.025234332), (2, 0.95061374)]
doc :  13 [(0, 0.06377683), (1, 0.880449), (2, 0.05577419)]
doc :  14 [(0, 0.85502875), (1, 0.06916985), (2, 0.0758014)]
doc :  15 [(0, 0.8999522), (1, 0.048461504), (2, 0.051586293)]
doc :  16 [(0, 0.72

doc :  500 [(0, 0.73098505), (1, 0.09087687), (2, 0.17813808)]
doc :  501 [(0, 0.09197015), (1, 0.069229096), (2, 0.8388008)]
doc :  502 [(0, 0.8998655), (1, 0.05066353), (2, 0.049470995)]
doc :  503 [(0, 0.862012), (1, 0.06912384), (2, 0.068864085)]
doc :  504 [(0, 0.5792644), (1, 0.1607444), (2, 0.25999117)]
doc :  505 [(0, 0.8482265), (1, 0.09225434), (2, 0.059519142)]
doc :  506 [(0, 0.07763433), (1, 0.08148837), (2, 0.84087735)]
doc :  507 [(0, 0.6699769), (1, 0.2321339), (2, 0.09788923)]
doc :  508 [(0, 0.87910634), (1, 0.05059708), (2, 0.07029654)]
doc :  509 [(0, 0.058509424), (1, 0.883137), (2, 0.058353618)]
doc :  510 [(0, 0.862093), (1, 0.067064725), (2, 0.070842355)]
doc :  511 [(0, 0.05365984), (1, 0.041129082), (2, 0.90521103)]
doc :  512 [(0, 0.84854954), (1, 0.082618184), (2, 0.06883226)]
doc :  513 [(0, 0.09199571), (1, 0.77499336), (2, 0.13301086)]
doc :  514 [(0, 0.22798136), (1, 0.11003817), (2, 0.66198045)]
doc :  515 [(0, 0.6924143), (1, 0.20201471), (2, 0.1055709

In [51]:
lstTopicsCorpus[0:5]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[1, 0, 1, 2, 0]

**In this section we are merging the text corpus with the topic. So all the text rows topic will be tied up in one dataset**

In [52]:
df_Topic = pd.DataFrame(
    {'Text': corpus,
     'Cleaned_Text': clean_corpus,
     'Topic': lstTopicsCorpus
    })

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
df_Topic[:5]

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Text,Cleaned_Text,Topic
0,Insulin NPH Human [NOVOLIN N] 100 unit/mL susp...,"[unit, suspension, subcutaneous, direct]",1
1,"Patient arrives ambulatory, Gait steady, Hist...","[arrive, ambulatory, gait, steady, history, ob...",0
2,"Peripheral IV site, established in the right ...","[site, establish, right, forearm, use, gauge, ...",1
3,No: new confusion or inability to stay alert ...,"[new, confusion, inability, stay, alert, awake...",2
4,Spent 15 minutes with the patient and greater ...,"[spend, minute, patient, great, %, time, spend...",0


## Sentence Similarity Check

We will pick one sentence from one topic and check similarity among all the sentences

Try to find out where the test sentence is more similar

In [58]:
from sentence_transformers import SentenceTransformer

model_p = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
# Getting the embedding
q_emb = model_p.encode(df_Topic['Text'].tolist()[0])
p_emb = model_p.encode(df_Topic['Text'].tolist())

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Create a function to display the statistics

In [61]:
from sentence_transformers import  util

def getSimiTopic(ind,p_emb):
    # Getting the embedding
    q_emb = model_p.encode(df_Topic['Text'].tolist()[ind])
    #p_emb = model_p.encode(df_Topic['Text'].tolist())
    
    print(df_Topic['Text'].tolist()[ind])
    
    # Printing the similarity score 
    sim_score = util.pytorch_cos_sim(q_emb, p_emb).cpu().detach().numpy()[0]
    
    # Data frame create
    df_Score = pd.DataFrame(
    {'Sim': sim_score,
     'Topic': lstTopicsCorpus
    })
    
    #display(df_Score.groupby(['Topic']).agg({"Sim": ['mean','sum','min','max','count']}))
    return df_Score.groupby(['Topic']).agg({"Sim": ['mean','sum','min','max','count']})

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
def getTopicScore(topic, p_emb):
    # We need a list of dataframes for each topic 
    lst_df  = []
    
    # Get the list of index in the dataframe for the selected topic
    lst_ind = df_Topic.loc[df_Topic['Topic'] == topic].index.tolist()
    
    for ind in lst_ind:
        print("added", ind)
        df = getSimiTopic(ind,p_emb)
        lst_df.append(df)
    return pd.concat(lst_df).groupby(level=0).mean()


/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
# topic 0
getTopicScore(0,p_emb)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


added 1
 Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Skin warm.
added 4
Spent 15 minutes with the patient and greater than 50% of this time was spent counseling the patient regarding diagnosis and available treatment options.
added 14
 Explained diagnosis and treatment plan; patient expressed understanding of the content, Patient was given a copy of this note
added 15
 Patient arrives, via stretcher, via Emergency Medical Services, Unsteady gait, Lift to cart, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Oriented to person, place and time.
added 16
 Dimension 3 Emotional, Behavioral or Cognitive Conditions and Complications: The patient received a Risk Score of 1. Interference with Addiction Recovery Emotional concerns related to negative consequences and effects of addiction; patient is able to view these as part of addiction and recovery.
added 17
 Identifie

 Identified Illness as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Verbal Instructions, Patient verbalized an understanding of discharge teaching.
added 137
The patient has no contraindications to the vaccine as documented in the CDC Vaccine Information Statement.
added 139
 I independently interviewed and examined the patient and agree with the history, exam, assessment and plan of care.
added 141
Plan is to continue working towards work hardening goals of 50 lb occasional weight handling.
added 151
 History obtained from patient, Patient cooperative, alert, Oriented to person, place and time.
added 153
Patient was identified using two identifiers and permission was obtained from the patient to perform a fingerstick INR.
added 157
Explained diagnosis and treatment plan; patient/child/caregiver expressed understanding of the content.Caller agree

 Identified Injury as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Patient verbalized an understanding of discharge teaching.
added 230
 Complex assessment performed, Patient arrives ambulatory, Gait steady, History obtained from patient, Patient appears, in distress due to pain, Patient cooperative, alert, Oriented to person, place and time.
added 231
 The patient had a chance to have any questions about this procedure answered, understand(s) and wish(es) to proceed.
added 233
The patient understands the information and questions answered; the patient wishes to proceed with the biopsy.
added 234
The diagnosis and treatment plans were explained and the patient expressed understanding of the content.
added 237
 Patient discharged to home, ambulating without assistance, Discharge instructions given to patient, Above pe

 Identified Illness as a learning need, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Verbal Instructions, Patient verbalized an understanding of discharge teaching.
added 314
Patient is here for the following immunization(s):  Inactivated Influenza Virus Vaccination; Tetanus and Diphtheria and Acellular Pertussis (Tdap) Vaccine
added 317
 Patient discharged to home, ambulating without assistance, driving self, unaccompanied, Above person(s) verbalized understanding of discharge instructions and follow-up care.
added 319
Explained diagnosis and treatment plan; patient/parents expressed understanding of the content.
added 322
 PATIENT EDUCATION:  Ready to learn, no apparent learning barriers were identified; learning preferences include listening.
added 324
 Disp : 100 - TAB, Directions : ONE EVERY BEDTIME, Refill : 3 Time(s), Expire Rx on : 1 YR, Route : ORAL
added 326
 Patient arrives, via stretcher,

added 461
 Patient arrives, via Emergency Medical Services, History obtained from patient, Patient appears comfortable, Patient cooperative, alert, Oriented to person, place and time.
added 465
 Patient currently uses tobacco, smokes cigarettes, smokes 1/2 packs per day, Patient denies alcohol use.
added 472
Patient will demonstrate and/or verbalize understanding of home exercise program in 6 sessions.
added 474
 Family ready to learn, no apparent learning barriers were identified; learning preferences include listening.
added 475
Description: The patient was seen for an individual session today to review progress in treatment and any changes made to the treatment plan.
added 477
Pain Scale-The Numeric pain scale was used to identify the patient's level of pain (0= no pain and 10 = worst possible pain).
added 479
I have personally interviewed and examined the patient and reviewed the medical history with the patient.
added 480
 Patient arrives, via hospital wheelchair, Gait steady, His

 Identified Illness as a learning need, Patients primary language is English, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Verbal Instructions.
added 577
 Client was provided with verbal and written (  Weekend Planning,   &  ) instructions.
added 578
 Patient discharged to home, ambulating without assistance, family driving, accompanied by parent, Discharge instructions given to patient, Above person(s) verbalized understanding of discharge instructions and follow-up care.
added 580
The client verbalized understanding and consented to the plan of care and the goals established.
added 582
Patient will demonstrate and/or verbalize understanding of home exercise program in 1 sessions for improved self management of the condition.
added 585
Patient is here for the following immunization(s):  Live Intranasal Influenza Virus Vaccination
added 587
 Identified Illness as a learn

 Ready to learn, no apparent learning barriers were identified; learning preferences included listening.
added 704
 Identified Injury as a learning need, Identified Follow-up care as a learning need, No Barriers to learning were identified, No interventions were used to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Verbal Instructions, Patient verbalized an understanding of discharge teaching.
added 705
 Identified Illness as a learning need, Identified Follow-up care as a learning need, No Barriers to learning were identified, Involved Family Member or Primary Caregiver to address Barriers to Learning, Teaching methods used included Printed Patient Instructions, Verbal Instructions, Patient verbalized an understanding of discharge teaching.
added 707
The rationale for hand therapy intervention was discussed and patient agrees to above plan of care.
added 708
No: new deformity other than swelling; a grating or crackling sound or sensation on

Sim                                      
           mean         sum       min       max count
Topic                                                
0      0.444369  163.972214  0.003560  1.000000   369
1      0.299089   62.808781  0.036853  0.544586   210
2      0.242859   41.528893  0.026311  0.570498   171

In [66]:
getTopicScore(1,p_emb)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


added 0
Insulin NPH Human [NOVOLIN N] 100 unit/mL suspension subcutaneous as directed by prescriber.
added 2
 Peripheral IV site, established in the right forearm, using an 18 gauge catheter, in one attempt.
added 8
 Discussed the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 10
 Given current medication regimen, the following parameters should be monitored by outpatient providers: None
added 11
 The risks and benefits of the procedure were discussed, and the patient consented to this procedure.
added 13
 Discussed goals, risks, alternatives, advanced directives, and the necessity of other members of the surgical team participating in the procedure with the patient.
added 19
 Discussed risks, goals, and necessity of other members of the team participating in the procedure with patient.
added 24
 No: joint swelling; pain of a type other than joint pain; a limp without known injury; discomfort in legs that improves with mov

No: new confusion or inability to stay alert and awake; severe lethargy or floppiness; refusing to move the neck; current or recent seizure; high-pitched cry (like a cat's cry) OR a weak whimper or moaning cry that is not consolable; purple or red rash/blotches that stay when pressed by a glass (purpuric rash) or bulging or tense fontanel (soft spot on the head) when not crying
added 219
Albuterol Aerosol 90 mcg/Actuation 2 puffs by inhalation as directed by prescriber as needed.
added 221
 Discussed acne and treatment in detail along with consent with risks, alternatives, and benefits.
added 222
Albuterol [PROVENTIL/VENTOLIN] 90 mcg/Act HFA Aerosol 1-2 puffs by inhalation every 4 hours as needed.
added 228
 Discussed advance directives and the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 229
 Cyanocobalamin [VITAMIN B12] 1,000 mcg/mL solution 1,000 micrograms intramuscular as directed by prescriber.
added 236
The indivi

 Discussed risks, goals, alternatives, and advanced directives, the necessity of other members of the healthcare team participating in the procedure with the patient (or legal representative and others present during the discussion).
added 433
 After discussion of the risks, benefits, and alternatives to treatment with cryotherapy, informed consent was obtained.
added 436
Return here or go to the nearest Emergency Department if you notice any of the problems listed below or you have other concerns.
added 445
 Discussed the risks, benefits, alternatives, and the necessity of other members of the healthcare team, both male and female, participating in the procedure.
added 462
 Discussed risks, goals, alternatives, and advanced directives, the necessity of other members of the healthcare team participating in the procedure with the patient (or legal representative and others present during the discussion).
added 464
 Patient's age is 9 years of age or older: Administer Inactivated Influen

Ipratropium-Albuterol [COMBIVENT] 18-103 mcg/Actuation Aerosol 2 puffs by inhalation four times a day.
added 702
 Discussed the risks, benefits, alternatives, and the necessity of other members of the healthcare team participating in the procedure.
added 703
 Discussed with the patient the necessity of other members of the healthcare team, both male and female, participating in the procedure if needed.
added 706
 Discussed risks, goals, alternatives, advanced directives and the necessity of other members of the healthcare team participating in the procedure.
added 713
 Mother is blood type B+, is Hepatitis B negative, HIV negative, and was found to be GBS negative.
added 715
Fluticasone Propionate [FLONASE] 50 mcg/Act Aerosol 2 sprays nasally one time daily as needed.
added 717
Albuterol [PROVENTIL] 0.083 % Neb solution 3 mL by inhalation as directed by prescriber as needed.
added 718
 Disp : 150 ml - Suspension, Sig : Give 5 ml by mouth 3 times a day for 10 days, Refill : None.
added 

Sim                                      
           mean         sum       min       max count
Topic                                                
0      0.299089  110.363998 -0.019340  0.577959   369
1      0.301583   63.332531 -0.008422  1.000000   210
2      0.208212   35.604259 -0.028494  0.510417   171

In [67]:
getTopicScore(2,p_emb)

/home/dell/.virtualenvs/event_part/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


added 3
 No: new confusion or inability to stay alert and awake; currently struggling to breathe, even while inactive or resting; currently feeling like you are going to collapse every time you stand (sit); vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
added 5
No: new confusion or inability to stay alert and awake; newly painful neck with difficulty bending the neck; paralyzed face muscles or severe dizziness
added 6
 No: new confusion or inability to stay alert and awake; struggling to breathe even while inactive or resting; currently feeling like you are going to collapse every time you stand or sit up; vomit that looks like ground coffee; vomiting blood; uncontrollable or continuous rectal bleeding; black, sticky, tar-like stools; heavy vaginal bleeding or purple or red rash/blotches that stay whe

added 201
 Patient requires extensive assistance in the following activities:  bathing, dressing, toileting, transfer to/from bed/chair, mobility.
added 213
 No: new confusion or inability to stay alert and awake; noisy, wheezy or raspy breathing that does not clear with coughing; newly stiff or painful neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash); currently feeling like you are going to collapse every time you stand (sit); muffled voice or inability to open mouth fully or complete inability to swallow
added 220
 No: new confusion or inability to stay alert and awake; noisy, wheezy or raspy breathing that does not clear with coughing; newly stiff or painful neck; purple or red rash/blotches that stay when pressed by a glass (purpural rash); currently feeling like you are going to collapse every time you stand (sit); muffled voice or inability to open mouth fully or complete inability to swallow
added 225
No: new confusion or inability to stay aler

 No: new confusion or inability to stay alert and awake; new shortness of breath; new wheezing or chest tightness; sudden swelling of the lips, tongue, or mouth; sudden onset of trouble swallowing or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
added 423
No: new confusion or inability to stay alert and awake or purple or red rash/blotches that stay when pressed by a glass (purpural rash)
added 426
 D. Ongoing assessment and treatment by the multidisciplinary treatment team, including disposition planning, is underway.
added 428
 No: inability to speak or make normal sounds; new confusion or inability to stay alert and awake; sudden swelling of the lips, tongue, or mouth; currently struggling to breathe, even while inactive or resting; abrupt onset of breathing problems that came over the course of a minute or two; sudden onset of cough, choking or gagging after inhaling something into your airway; pain, pressure or tightness in the chest, jaw or arm; ch

added 613
Dimension 2 Biomedical Conditions and Complications: The patient received a Risk Score of 1. Patient demonstrates adequate ability to tolerate and cope with physical discomfort.
added 622
 She denies any signs or symptoms of bruising, bleeding complications, or thromboembolic events.
added 628
 The following should be done yearly:height, weight, visual screening and blood pressure: done this year  Injury prevention is as follows: seatbelts: yes Smoke detectors: yes  Bicycle or motorcycle helmet no (PLEASE WEAR HELMETS) Guns and violence: yes and there are no guns in the home  Other risk factors reviewed:  passive smoke: not at risk Exposure to tuberculosis: no exposure   Sunscreens: yes Brushing teeth and dental care: yes, you see a dentist regularly.
added 629
No: currently inconsolable crying; vomiting; diarrhea; not feeding normally; fever (rectal temperature) of more than 100.4
added 634
 No: localized area of red, swollen, tender and warm skin that is expanding; red stre

Sim                                     
           mean        sum       min       max count
Topic                                               
0      0.242859  89.614975 -0.032125  0.548266   369
1      0.208212  43.724529 -0.027127  0.563039   210
2      0.323876  55.382877 -0.017503  1.000000   171